# Training and A/B test for Conversion agents - <font color='blue'> Proportion</font> of sale

In [6]:
import gym, recogym
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import pandas as pd
import pickle as pkl
pd.options.mode.chained_assignment = None 
# from inspect import getsource
from recogym.evaluate_agent_sale import verify_agents_sale, display_metrics, verify_agents_sale_extended
from tqdm import tqdm

# env_1_sale_args is a dictionary of default parameters (i.e. number of products)
from recogym import env_1_sale_args, Configuration
from recogym.agents.sale_agent import train_agents, train_timeagents
from recogym.envs.utils_sale import format_avg_result, avg_result, format_avg_result_extended, avg_result_extended 

# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 0
env_1_sale_args['num_products'] = 10
env_1_sale_args['number_of_flips'] = 10 

num_products = env_1_sale_args['num_products']
print('Number of products =',num_products)
print('Number of flips =',env_1_sale_args['number_of_flips'])
nb_flips = env_1_sale_args['number_of_flips']

# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 42
env_1_sale_args['mu_sale'] = False 

# env_1_sale_args['kappa'] = 0.5
print('Value of kappa =',env_1_sale_args['kappa'])

# Initialize the gym 
env = gym.make('reco-gym-sale-v1')
env.init_gym(env_1_sale_args)



Number of products = 10
Number of flips = 10
Value of kappa = 0.2


In [7]:
# Repository to save pickles
data_repo = 'data_conversion/'

## Train baseline agents

#### Settings

In [8]:
## Choose number of users for training and AB test
# Number of users for the training
env_1_sale_args['num_users'] = 6 ##tochange !!
# env_1_sale_args['num_users'] = 5000
num_users = env_1_sale_args['num_users']

# Number of users for the A/B test
env_1_sale_args['num_users_AB'] = 7 ##tochange !!
# env_1_sale_args['num_users_AB'] = 5000 
num_users_AB = env_1_sale_args['num_users_AB']

# Choose user features
from recogym.agents.sale_agent import CountViewsClicksFeatureProvider, CountViewsFeatureProvider, ShareViewsClicksFeatureProvider, ShareViewsFeatureProvider
vc_feature = CountViewsClicksFeatureProvider(env.config)
v_feature = CountViewsFeatureProvider(env.config)
vc_share_feature = ShareViewsClicksFeatureProvider(env.config)
v_share_feature = ShareViewsFeatureProvider(env.config)
features = {'vc':vc_feature,
           'v':v_feature,
           'vc_share':vc_share_feature,
           'v_share':v_share_feature}
feature_name = 'v_share'
feature = features[feature_name]

#### <font color='red'> Number of A/B tests</font>

In [9]:
# # Choose number of A/B tests
num_AB_tests = 2 ##tochange !
# num_AB_tests = 25 ##tochange !

#### Logs

In [10]:
agents={}
logs={}

############## Random agent
name_agent = 'rand'+str(nb_flips)
from recogym.agents import RandomAgent, random_args
random_agent = RandomAgent(Configuration(random_args))
agents[name_agent] = random_agent


try:
    logs[name_agent] = pkl.load(open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','rb'))
    print('--- Logs loaded---')
except: 
    print("--- Generate logs ---")
    logs[name_agent] = deepcopy(env).generate_logs(num_users)
    print(data_repo + 'data' + str(num_users) + name_agent + '.pkl')
    pkl.dump(logs[name_agent], open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','wb'))

--- Logs loaded---


#### Training for the <font color='blue'> Proportion</font> of sale

##### No weights

In [11]:
name_extension = 'prop'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('timeagents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_timeagents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,linear_reg=True, repo = data_repo)

--- Trained agents loaded --- 


In [12]:
try:
    for i in range(num_AB_tests):
        name_extension = 'prop'+'_nb'+str(i)
        name = name = 'likelihood_saleclickprod_discountprop_full'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'prop'+'_nb'+str(i)
        res=verify_agents_sale_extended(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

--- A/B test results loaded ---


In [13]:
res_avg = avg_result_extended(res_dict)
(res_recap, res_recap_latex, 
 res_AB, res_AB_latex, 
 res_true, res_true_latex) = format_avg_result_extended(res_avg) #get dataframe & corresponding latex table

-- A/B test --


,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Rand,0.546 (0.134) %,1 (0),16.03 (8.45) %,6 (0),33.97 (0.00) %
1,PCS,0.957 (0.266) %,1 (0),19.02 (4.23) %,6 (1),27.99 (0.00) %
2,DPCS,0.693 (0.397) %,1 (0),16.03 (0.00) %,8 (1),36.96 (4.22) %


-- True metrics --


,Agent,True CTR,True PCS,True OS,True NCS,NDPC,DPCSO,DPCSN
0,Rand,0.593 (0.019) %,4.82 (0.36) %,3.90 (0.37) %,3.81 (0.50) %,0.573 (0.062) pm,0.110 (0.002) pm,0.120 (0.013) pm
1,PCS,0.718 (0.281) %,5.87 (0.63) %,4.65 (0.46) %,5.26 (0.88) %,0.861 (0.420) pm,0.180 (0.093) pm,0.081 (0.001) pm
2,DPCS,0.684 (0.025) %,5.01 (0.73) %,2.89 (0.98) %,3.84 (0.46) %,0.687 (0.125) pm,0.290 (0.023) pm,0.161 (0.043) pm


In [14]:
print(res_AB_latex+'\n')
print(res_true_latex)

\begin{tabular}{llllll}
\toprule
Agent &              CTR & Att Sales &          Att CR &  Sales &              CR \\
\midrule
 Rand &  0.546 (0.134) \% &     1 (0) &  16.03 (8.45) \% &  6 (0) &  33.97 (0.00) \% \\
  PCS &  0.957 (0.266) \% &     1 (0) &  19.02 (4.23) \% &  6 (1) &  27.99 (0.00) \% \\
 DPCS &  0.693 (0.397) \% &     1 (0) &  16.03 (0.00) \% &  8 (1) &  36.96 (4.22) \% \\
\bottomrule
\end{tabular}


\begin{tabular}{llllllll}
\toprule
Agent &         True CTR &       True PCS &        True OS &       True NCS &              NDPC &             DPCSO &             DPCSN \\
\midrule
 Rand &  0.593 (0.019) \% &  4.82 (0.36) \% &  3.90 (0.37) \% &  3.81 (0.50) \% &  0.573 (0.062) pm &  0.110 (0.002) pm &  0.120 (0.013) pm \\
  PCS &  0.718 (0.281) \% &  5.87 (0.63) \% &  4.65 (0.46) \% &  5.26 (0.88) \% &  0.861 (0.420) pm &  0.180 (0.093) pm &  0.081 (0.001) pm \\
 DPCS &  0.684 (0.025) \% &  5.01 (0.73) \% &  2.89 (0.98) \% &  3.84 (0.46) \% &  0.687 (0.125) pm &  0.290 (0.

##### Weights

In [15]:
name_extension = 'propweights'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('timeagents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_timeagents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,linear_reg=True, weights = True, repo = data_repo)

--- Trained agents loaded --- 


In [16]:
try:
    for i in range(num_AB_tests):
        name_extension = 'propweights'+'_nb'+str(i)
        name = 'likelihood_saleclickprod_discountpropweights_full'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'propweights'+'_nb'+str(i)
        res=verify_agents_sale_extended(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

--- A/B test results loaded ---


In [17]:
res_avg = avg_result_extended(res_dict)
(res_recap, res_recap_latex, 
 res_AB, res_AB_latex, 
 res_true, res_true_latex) = format_avg_result_extended(res_avg) #get dataframe & corresponding latex table

-- A/B test --


,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Rand,0.470 (0.004) %,0 (0),10.09 (8.40) %,4 (5),25.00 (21.13) %
1,PCS,0.580 (0.178) %,0 (0),10.09 (8.40) %,1 (1),16.03 (8.45) %
2,DPCS,0.822 (0.547) %,1 (0),16.03 (8.45) %,6 (1),30.98 (4.23) %


-- True metrics --


,Agent,True CTR,True PCS,True OS,True NCS,NDPC,DPCSO,DPCSN
0,Rand,0.606 (0.077) %,4.99 (1.66) %,3.89 (1.85) %,4.23 (1.95) %,0.591 (0.124) pm,0.135 (0.040) pm,0.094 (0.047) pm
1,PCS,0.663 (0.177) %,3.45 (1.62) %,1.92 (1.25) %,2.31 (1.33) %,0.429 (0.093) pm,0.196 (0.005) pm,0.146 (0.002) pm
2,DPCS,0.763 (0.064) %,4.33 (0.17) %,2.38 (0.06) %,3.26 (0.04) %,0.661 (0.029) pm,0.298 (0.008) pm,0.163 (0.006) pm


In [18]:
print(res_AB_latex+'\n')
print(res_true_latex)

\begin{tabular}{llllll}
\toprule
Agent &              CTR & Att Sales &          Att CR &  Sales &               CR \\
\midrule
 Rand &  0.470 (0.004) \% &     0 (0) &  10.09 (8.40) \% &  4 (5) &  25.00 (21.13) \% \\
  PCS &  0.580 (0.178) \% &     0 (0) &  10.09 (8.40) \% &  1 (1) &   16.03 (8.45) \% \\
 DPCS &  0.822 (0.547) \% &     1 (0) &  16.03 (8.45) \% &  6 (1) &   30.98 (4.23) \% \\
\bottomrule
\end{tabular}


\begin{tabular}{llllllll}
\toprule
Agent &         True CTR &       True PCS &        True OS &       True NCS &              NDPC &             DPCSO &             DPCSN \\
\midrule
 Rand &  0.606 (0.077) \% &  4.99 (1.66) \% &  3.89 (1.85) \% &  4.23 (1.95) \% &  0.591 (0.124) pm &  0.135 (0.040) pm &  0.094 (0.047) pm \\
  PCS &  0.663 (0.177) \% &  3.45 (1.62) \% &  1.92 (1.25) \% &  2.31 (1.33) \% &  0.429 (0.093) pm &  0.196 (0.005) pm &  0.146 (0.002) pm \\
 DPCS &  0.763 (0.064) \% &  4.33 (0.17) \% &  2.38 (0.06) \% &  3.26 (0.04) \% &  0.661 (0.029) pm &  0.298

In [19]:
res_avg

{'CTR':                                           Agent     0.025     0.500     0.975  \
 0                                        rand10  0.003301  0.009468  0.020643   
 1           likelihood_saleclickprodpropweights  0.003285  0.014123  0.037701   
 2  likelihood_saleclickprod_discountpropweights  0.013611  0.024177  0.039024   
 
        Mean       std        se  
 0  0.004704  0.000059  0.000042  
 1  0.005802  0.002519  0.001781  
 2  0.008224  0.007729  0.005465  ,
 'Tot sales ATT':                                           Agent  TotSalesAtt  Mean  std  \
 0                                        rand10            0   0.5  1.0   
 1           likelihood_saleclickprodpropweights            2   0.5  1.0   
 2  likelihood_saleclickprod_discountpropweights            3   1.0  1.0   
 
          se  
 0  0.707107  
 1  0.707107  
 2  0.707107  ,
 'Share user with sale ATT':                                           Agent     0.025     0.500     0.975  \
 0                          